# Creating a web scraper for Zillow.com
This scraper will search all of Zillow's listings for both recently sold and currently available single family homes in a specific state/county.

## Helpful videos:
https://www.youtube.com/watch?v=dRcvJRmqFHQ&ab_channel=WebScrapingwithAndy\
https://www.youtube.com/watch?v=a3Cuq2csLWk

## Helpful websites:
curl.trillworks.com\
https://www.scrapingdog.com/blog/scrape-zillow/

In [1]:
# Import libraries
from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# import chromedriver_autoinstaller
import requests
import pandas as pd
import os
import numpy as np
import regex as re
import time
import json

# from scrapingFunctions import siteType

In [2]:
# Initial variables
page = 0
size = 0
county = 'brevard'
state = 'fl'
results = []
# path = '/Users/jonathantiede/Documents/GitHub_Repositories/RealEstate_Recommender/chromedriver'
# chromedriver_autoinstaller.install()

In [3]:
# cookies = {
#     'zguid': '24|%246fb0b799-0e43-4f66-8d83-21d596fdcb10',
#     '_ga': 'GA1.2.276650241.1707089070',
#     'zjs_anonymous_id': '%226fb0b799-0e43-4f66-8d83-21d596fdcb10%22',
#     'zg_anonymous_id': '%2274af3d1f-88c7-47f1-8fba-2012fad9ebd0%22',
#     '_gcl_au': '1.1.34214727.1707089070',
#     '_pxvid': '8ba37f46-c3b4-11ee-9e13-8258276fc6fd',
#     '__pdst': '4643e3b9e1e247b593e735d4a1dc968c',
#     '_pin_unauth': 'dWlkPU9USmxaREptTkRFdE1UazFZUzAwWmpobUxXRTNaVGt0TkdabU56WTFaVEV6WW1RNA',
#     'g_state': '{"i_l":0}',
#     'loginmemento': '1|8a454d231ed7eca409e814a99a3597c00ceda63377d0b3bf92c6a65c545305f6',
#     'userid': 'X|3|2d7d88bcfa818bd7%7C2%7Cb4EgAuPxaVSpXpvRhJu7beBPJcglJyW6hchQYvmUSWA%3D',
#     'zjs_user_id': '%22X1-ZU11on5i9m0cg7d_8469x%22',
#     '_fbp': 'fb.1.1707264904428.264116798',
#     '__gads': 'ID=7ca70bfc69dc898a:T=1707264903:RT=1707269602:S=ALNI_MY6Wr18mXjDU2wlmX36axPlaKh8Kg',
#     '__gpi': 'UID=00000dbff955306f:T=1707264903:RT=1707269602:S=ALNI_MbHVNs2mJJeAPEBLbEwrfM16s39Xw',
#     '__eoi': 'ID=501fbd9f5a950fbd:T=1707264903:RT=1707269602:S=AA-AfjbGo0IdBShZNSjY3lVzM-pu',
#     '_derived_epik': 'dj0yJnU9bXZ4c1ZiSzMzaDdoTHkxcFVlS1I2M2tuYlByd3pWamEmbj1RdGYxQmRtT1VOa083Q3Ztam8tQ25nJm09NCZ0PUFBQUFBR1hFMDVJJnJtPTQmcnQ9QUFBQUFHWEUwNUkmc3A9Mg',
#     'search': '6|1709990036626%7Crect%3D28.215096444892627%2C-80.64248085021973%2C28.160627387683906%2C-80.76607704162598%26rid%3D19307%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D1%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26featuredMultiFamilyBuilding%3D0%26student-housing%3D0%26income-restricted-housing%3D0%26military-housing%3D0%26disabled-housing%3D0%26senior-housing%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%0919307%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09',
#     '_gid': 'GA1.2.1510494870.1707527650',
#     '_clck': '1ucal2l%7C2%7Cfj5%7C0%7C1495',
#     'zgsession': '1|ac91d803-6ced-4d61-af58-3344cf38c2df',
#     'ZILLOW_SID': '1|AAAAAVVbFRIBVVsVEl1oFA%2BlPUNlPy2CXA2%2BbQ6xlusmFtxm3icKJfu%2FNPdd6AwTUwRFP8nm8YoH%2BI6eYErO6IPePEMs%2BOSsFA',
#     'DoubleClickSession': 'true',
#     '_pxff_cc': 'U2FtZVNpdGU9TGF4Ow==',
#     'pxcts': '1a8059e7-c7b8-11ee-bc13-a48e59dc98ec',
#     '_pxff_cfp': '1',
#     '_pxff_bsco': '1',
#     '_hp2_ses_props.1215457233': '%7B%22ts%22%3A1707530402686%2C%22d%22%3A%22www.zillow.com%22%2C%22h%22%3A%22%2F%22%7D',
#     'JSESSIONID': '6B0C926417E931EDA38581D36E0A8273',
#     'x-amz-continuous-deployment-state': 'AYABeHeuIlTO68bv5bk6Pw2W0IgAPgACAAFEAB1kM2Jsa2Q0azB3azlvai5jbG91ZGZyb250Lm5ldAABRwAVRzA3MjU1NjcyMVRZRFY4RDcyVlpWAAEAAkNEABpDb29raWUAAACAAAAADIzB4hzhmiWlvbd0ywAw6bF5aCqQGQKlbOW4ttIdoclc1d45Z3Hop4ZvtZqnpUXDxVppVpho0wzRNXtoEcoSAgAAAAAMAAQAAAAAAAAAAAAAAAAAAJROVYc16ydrrkxO++H9NAz%2F%2F%2F%2F%2FAAAAAQAAAAAAAAAAAAAAAQAAAAwX0YsX8DWlCfyrmJufrGEUUwHOXsU4m7SoqoMBU4m7SoqoMA==',
#     '_gat': '1',
#     '_hp2_id.1215457233': '%7B%22userId%22%3A%221841226270994960%22%2C%22pageviewId%22%3A%221467006781959177%22%2C%22sessionId%22%3A%223607095151200097%22%2C%22identity%22%3A%22X1-ZU11on5i9m0cg7d_8469x%22%2C%22trackerVersion%22%3A%224.0%22%2C%22identityField%22%3Anull%2C%22isIdentified%22%3A1%7D',
#     '_uetsid': 'b24060e0c7b111ee86370d00e97ef46f',
#     '_uetvid': '8cd96630c3b411ee92972bee046f6165',
#     '_px3': 'e13baf1f084ae1709c67a30287a5688819599fb15b26454c397b52a9ca5cdd8f:ZjU8VMNRANwDF5NtxuT0GGNMoeJdqGIGl1kF6jA2J/1RY6vFqBfcjL9DdTbbO0Ztr6Tz83JYnHhpBqOJ4s4jog==:1000:MmcjjDkTJOwjAQXlnrD2Vh/bcNGUV/gxRezs77+lrnI5FiKWgyu1djD1t+/HDk8CeXp/muYGpe3kjZVgm3aAAdKAYFJi/WKLKGB2V6vnoyKbZXwQYwwzbapoLyxJLaouYkB4gJyn2d2Y7WRf6Y7+CFsz92iIazmitSxXfj4Z1BJxP0lZufl6nM9oN3DJhhTKx2lN0LxtkF6a/hAd6Chw1unoORKzZrXvVPEVk7mcoAY=',
#     '_clsk': 'es802d%7C1707530465456%7C4%7C0%7Cy.clarity.ms%2Fcollect',
#     'AWSALB': 'Z4yc6RK/dkjbWRXT0tWoSMvoMe67cP64CZo2hUFm2Ni3ZXeo8hC7fTO/GB8L0O68xJElr8NbvBbILnBahKSlRsPUVc1Eu3TPtsN70LrndNLPVWhtCmWU5eHltI4t',
#     'AWSALBCORS': 'Z4yc6RK/dkjbWRXT0tWoSMvoMe67cP64CZo2hUFm2Ni3ZXeo8hC7fTO/GB8L0O68xJElr8NbvBbILnBahKSlRsPUVc1Eu3TPtsN70LrndNLPVWhtCmWU5eHltI4t',
# }

# headers = {
#     'authority': 'www.zillow.com',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'accept-language': 'en-US,en;q=0.9',
#     'referer': 'https://www.zillow.com/',
#     'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"macOS"',
#     'sec-fetch-dest': 'document',
#     'sec-fetch-mode': 'navigate',
#     'sec-fetch-site': 'same-origin',
#     'sec-fetch-user': '?1',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
# }

# params = {
#     'searchQueryState': '{"filterState":{"isTownhouse":{"value":true},"isLotLand":{"value":true},"isManufactured":{"value":true},"isMultiFamily":{"value":true},"isApartment":{"value":true},"isCondo":{"value":true},"isSingleFamily":{"value":true},"isApartmentOrCondo":{"value":true},"isRecentlySold":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isPreMarketForeclosure":{"value":false},"isForRent":{"value":false},"isForSaleByAgent":{"value":true},"isForSaleByOwner":{"value":true},"isAuction":{"value":true},"isComingSoon":{"value":true},"isForSaleForeclosure":{"value":true},"isNewConstruction":{"value":true},"sortSelection":{"value":"globalrelevanceex"}},"regionSelection":[{"regionId":1556}],"usersSearchTerm":"Brevard County FL homes"}',
# }

# response = requests.get('https://www.zillow.com/homes/', params=params, cookies=cookies, headers=headers)

headers = {
    'authority': 'www.zillow.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'referer': 'https://www.zillow.com/',
    'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    }

# url = 'https://www.zillow.com/%s-county-%s/' %(county, state)
# cService = webdriver.ChromeService(executable_path=path)
# options = webdriver.ChromeOptions()
# options.add_argument("start-maximized")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])
# options.add_experimental_option('useAutomationExtension', False)
# driver = webdriver.Chrome(service=cService, options=options)
# url = 'https://www.realtor.com'
# driver.get(url)
# time.sleep(5)
# html = driver.find_element(By.XPATH,'/html')
# html.send_keys(Keys.END)
# time.sleep(5)
# response = driver.page_source
# driver.close()


# The code below is used ONLY to initialize the status code to 200 for the while loop in the next code box. There has to be a better way
# to initialize or loop...
url = 'https://www.zillow.com/%s-county-%s/' %(county, state)
response = requests.get(url=url, headers=headers)
time.sleep(0.100)
print(response)

<Response [200]>


In [4]:
page = 0
while response.status_code == 200:
    page += 1
    print('Scraping page: %i' %page)

    url = 'https://www.zillow.com/%s-county-%s/' %(county, state)
    
    params = {
    'searchQueryState': '{"pagination":{"currentPage":%s},"isMapVisible":true,"mapBounds":{"west":-81.67492903125,"east":-79.69738996875,"south":27.974820595781136,"north":28.639802399071872},"mapZoom":9,"regionSelection":[{"regionId":1556,"regionType":4}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true}' %page
    }
    # {"pagination":{"currentPage":%s},"isMapVisible":false,"mapBounds":{"west":-81.67492903125,"east":-79.69738996875,"south":27.568978152602924,"north":29.04158767448029},"mapZoom":9,"usersSearchTerm":"Brevard County, FL","regionSelection":[{"regionId":1556,"regionType":4}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true}
    # {"pagination":{"currentPage":%s},"isMapVisible":true,"mapBounds":{"west":-81.67492903125,"east":-79.69738996875,"south":27.974820595781136,"north":28.639802399071872},"mapZoom":9,"regionSelection":[{"regionId":1556,"regionType":4}],"filterState":{"sort":{"value":"globalrelevanceex"},"ah":{"value":true}},"isListVisible":true}

    response = requests.get(url=url, params=params, headers=headers)
    time.sleep(1)
    if response.status_code != 200:
        print('Last page found: %i' %(page-1))
        print('Zillow may have caught on to our scraping.')
        break
    
    content = BeautifulSoup(response.text, 'lxml')

    deck = content.find('ul', {'class':'List-c11n-8-84-3__sc-1smrmqp-0 StyledSearchListWrapper-srp__sc-1ieen0c-0 doa-doM gKnRas photo-cards photo-cards_extra-attribution'})
    # If you inspect the "deck" variable, print(deck.prettify()), you will see a great many "timeouts". Not sure how to fix this...

    if deck == None:
        print('Page %s deck contents are empty.' %page)
        break

    size = 0
    for card in deck.contents:
        size += 1
        script = card.find('script', {'type':'application/ld+json'}) # Excluding any advertisement cards. Script = None on ad cards.

        if script:
            script_json = json.loads(script.contents[0])
            
            try:
                square_feet = script_json['floorSize']['value']
            except:
                square_feet = ''

            try:
                latitude = script_json['geo']['latitude']
            except:
                latitude = ''
            
            try:
                longitude = script_json['geo']['longitude']
            except:
                longitude = ''
            
            try:
                address = script_json['name']
            except:
                address = ''
            
            try:
                homeDetailsURL = script_json['url']
            except:
                homeDetailsURL = ''

            try:
                price = card.find('span', {'class':'PropertyCardWrapper__StyledPriceLine-srp__sc-16e8gqd-1 iMKTKr'}).text
            except:
                price = ''

            try:
                beds = re.split('bds|ba| ', card.find('ul', {'class':'StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID'}).text)[0]
            except:
                beds = ''

            try:
                baths = re.split('bds|ba| ', card.find('ul', {'class':'StyledPropertyCardHomeDetailsList-c11n-8-84-3__sc-1xvdaej-0 eYPFID'}).text)[2]
            except:
                baths = ''

            results.append({
                'square_feet':square_feet,
                'latitude':latitude,
                'longitude':longitude,
                'address':address,
                'url':homeDetailsURL,
                'price':price,
                'beds':beds,
                'baths':baths
            })

    time.sleep(2)

df_housingInfo = pd.DataFrame.from_dict(results)

# Still need to figure out the timeout issue...
# Now you need to loop through all the home detail urls and add data to dataframe (age, zestimate, days on zillow, view, saves, has HOA )

Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Page 18 deck contents are empty.


In [5]:
df_housingInfo

,square_feet,latitude,longitude,address,url,price,beds,baths
0,"2,234",28.143036,-80.708885,"4945 Laguna Vista Dr, Melbourne, FL 32934",https://www.zillow.com/homedetails/4945-Laguna...,"$565,000",4,3
1,"1,915",28.161730,-80.651695,"3430 Sandpiper Ct, Melbourne, FL 32935",https://www.zillow.com/homedetails/3430-Sandpi...,"$289,900",3,3
2,"1,398",28.012882,-80.690780,"749 Fernandina St NW, Palm Bay, FL 32907",https://www.zillow.com/homedetails/749-Fernand...,"$285,000",3,2
3,"1,120",27.984524,-80.655075,"670 Bluefields St SE, Palm Bay, FL 32909",https://www.zillow.com/homedetails/670-Bluefie...,"$189,900",3,2
4,"1,984",28.471148,-80.802010,"6425 Stillwater Ave, Cocoa, FL 32927",https://www.zillow.com/homedetails/6425-Stillw...,$125K,4,2
...,...,...,...,...,...,...,...,...
148,"1,948",28.067455,-80.648690,"653 Brockton Way, West Melbourne, FL 32904",https://www.zillow.com/homedetails/653-Brockto...,"$415,000",3,2
149,"1,257",28.604168,-80.810450,"1033 1st Ave, Titusville, FL 32780",https://www.zillow.com/homedetails/1033-1st-Av...,"$339,900",3,2
150,"1,677",28.022417,-80.609510,"2140 Spring Creek Cir NE, Palm Bay, FL 32905",https://www.zillow.com/homedetails/2140-Spring...,"$344,900",3,2
151,"1,514",28.242783,-80.685410,"853 Villa Dr, Melbourne, FL 32940",https://www.zillow.com/homedetails/853-Villa-D...,"$399,000",3,2


In [6]:
# Update headers and cookies required for url get request on zillow "pop up" home details pages.
# It does not appear that these need to be updated for different home details urls. Should be a one and done for all searches.

headers = {
    'authority': 'www.zillow.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'referer': 'https://www.zillow.com/',
    'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
}

cookies = {
    'zguid': '24|%246fb0b799-0e43-4f66-8d83-21d596fdcb10',
    '_ga': 'GA1.2.276650241.1707089070',
    '_gid': 'GA1.2.650428496.1707089070',
    'zjs_anonymous_id': '%226fb0b799-0e43-4f66-8d83-21d596fdcb10%22',
    'zg_anonymous_id': '%2274af3d1f-88c7-47f1-8fba-2012fad9ebd0%22',
    '_gcl_au': '1.1.34214727.1707089070',
    '_pxvid': '8ba37f46-c3b4-11ee-9e13-8258276fc6fd',
    '__pdst': '4643e3b9e1e247b593e735d4a1dc968c',
    '_pin_unauth': 'dWlkPU9USmxaREptTkRFdE1UazFZUzAwWmpobUxXRTNaVGt0TkdabU56WTFaVEV6WW1RNA',
    'g_state': '{"i_l":0}',
    'loginmemento': '1|8a454d231ed7eca409e814a99a3597c00ceda63377d0b3bf92c6a65c545305f6',
    'userid': 'X|3|2d7d88bcfa818bd7%7C2%7Cb4EgAuPxaVSpXpvRhJu7beBPJcglJyW6hchQYvmUSWA%3D',
    'zjs_user_id': '%22X1-ZU11on5i9m0cg7d_8469x%22',
    'zgsession': '1|3b9257ba-347c-4841-b36c-fdc3456dc54a',
    'DoubleClickSession': 'true',
    'pxcts': 'be168d07-c47a-11ee-887d-93606b21f946',
    '_derived_epik': 'dj0yJnU9aU1rbm1vODhtUkZMZS1zVGt6V2ZvZkprXzhfXzdMUDkmbj03TDhoYXNGczNqMlJpMzVqR1hObGhnJm09NCZ0PUFBQUFBR1hCY1NZJnJtPTQmcnQ9QUFBQUFHWEJjU1kmc3A9NA',
    '_clck': '1ucal2l%7C2%7Cfj2%7C0%7C1495',
    '_fbp': 'fb.1.1707264904428.264116798',
    '__gads': 'ID=7ca70bfc69dc898a:T=1707264903:RT=1707269602:S=ALNI_MY6Wr18mXjDU2wlmX36axPlaKh8Kg',
    '__gpi': 'UID=00000dbff955306f:T=1707264903:RT=1707269602:S=ALNI_MbHVNs2mJJeAPEBLbEwrfM16s39Xw',
    '__eoi': 'ID=501fbd9f5a950fbd:T=1707264903:RT=1707269602:S=AA-AfjbGo0IdBShZNSjY3lVzM-pu',
    'ZILLOW_SID': '1|AAAAAVVbFRIBVVsVEuqA4d1HcFzglKcT9vnf%2FfrQt8v78O0uFl6Mzqi%2B%2FYkfJyktTrUhkn4f9g4k9YD3yJ8s0U2XrPMM2KTgmA',
    '_pxff_cc': 'U2FtZVNpdGU9TGF4Ow==',
    '_pxff_cfp': '1',
    '_pxff_bsco': '1',
    '_hp2_id.1215457233': '%7B%22userId%22%3A%221841226270994960%22%2C%22pageviewId%22%3A%22969084944458639%22%2C%22sessionId%22%3A%223445744565095364%22%2C%22identity%22%3A%22X1-ZU11on5i9m0cg7d_8469x%22%2C%22trackerVersion%22%3A%224.0%22%2C%22identityField%22%3Anull%2C%22isIdentified%22%3A1%7D',
    '_hp2_ses_props.1215457233': '%7B%22ts%22%3A1707344036844%2C%22d%22%3A%22www.zillow.com%22%2C%22h%22%3A%22%2F%22%7D',
    'JSESSIONID': '6615BEDA1FBA067AE6A883DE8FECB5B8',
    '_pxff_tm': '1',
    'search': '6|1709936081727%7Crect%3D29.581642766431106%2C-76.731081375%2C27.018581702681022%2C-84.641237625%26rid%3D1556%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26z%3D1%26listPriceActive%3D1%26fs%3D1%26fr%3D0%26mmm%3D0%26rs%3D0%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26featuredMultiFamilyBuilding%3D0%26student-housing%3D0%26income-restricted-housing%3D0%26military-housing%3D0%26disabled-housing%3D0%26senior-housing%3D0%26commuteMode%3Ddriving%26commuteTimeOfDay%3Dnow%09%091556%09%7B%22isList%22%3Atrue%2C%22isMap%22%3Atrue%7D%09%09%09%09%09',
    '_uetsid': '8cd95cd0c3b411ee83fd6fff755a5145',
    '_uetvid': '8cd96630c3b411ee92972bee046f6165',
    '_px3': 'fd31d6c20e31149e8b7e2ac80c843ba4073883cf07a4c65e5b92671c08279c9c:J8X+8iNDY6A9GZsgydWtAzObXkltNZFuKZeIJR2OLyIlwG76t3TzyhuK/JWpIH9POugLUa1E/4nXxfrzN8gzaA==:1000:TcE9jzmXIziDfX+sDL7JS/u0n6LHs8HyN+99S2V1CHW2loSVULIxyUBM9HP04XnTVHtZa70Fg7iHDhDkU+hKe1zErVuC8M5PdA7zNvFQGUNJLKFqp0aVJoaa0MarRkiBlzRES8KI2h2Gmy91XRHyOW1JJ1+rlv+7MJMKjuPfPyqI9pj0mzrhL0xfHvNYSNJgsyDkXZabV218n9cHpJ89DeXk3iLG8JVHHo8yvnkRxPE=',
    '_gat': '1',
    '_clsk': 'x4c9ar%7C1707344101927%7C9%7C0%7Cy.clarity.ms%2Fcollect',
    'AWSALB': 'fazhWvFSC1Q3cRGpAgbIMkDjK4IN422/fMCyBAiZ5Wkp9rpojXc2OhkfXDLR2wUseYC8FHvfzTn3xKfEzTnVL2+8AV47xAVIiEkLfaPGdFxySyZ/5EHL1XXNI5S7',
    'AWSALBCORS': 'fazhWvFSC1Q3cRGpAgbIMkDjK4IN422/fMCyBAiZ5Wkp9rpojXc2OhkfXDLR2wUseYC8FHvfzTn3xKfEzTnVL2+8AV47xAVIiEkLfaPGdFxySyZ/5EHL1XXNI5S7',
}

In [7]:
'''
url = 'https://www.zillow.com/homedetails/6425-Stillwater-Ave-Cocoa-FL-32927/43393692_zpid/'
response = requests.get(url=url, cookies=cookies, headers=headers)
content = BeautifulSoup(response.text, 'lxml')
details = content.find('div',{'class':'layout-content-container'})
print(details)
# days = details.find('dl',{'class':'OverviewStatsComponents__StyledOverviewStats-sc-7d6bsa-0 bOcuTH'})
# print(days)
'''

"\nurl = 'https://www.zillow.com/homedetails/6425-Stillwater-Ave-Cocoa-FL-32927/43393692_zpid/'\nresponse = requests.get(url=url, cookies=cookies, headers=headers)\ncontent = BeautifulSoup(response.text, 'lxml')\ndetails = content.find('div',{'class':'layout-content-container'})\nprint(details)\n# days = details.find('dl',{'class':'OverviewStatsComponents__StyledOverviewStats-sc-7d6bsa-0 bOcuTH'})\n# print(days)\n"

In [8]:
def siteType(tag):
    # So far I have found two versions of the main content section of the home details urls. This will account for both of them.
    try:
        details = tag.find('div',{'class':'layout-content-container'})
        # errorCheck = details.find_all('dt')[0].text
        print('Site type 1 found.')
    except AttributeError:
        details = tag.find('div',{'class':'data-column-container'})
        # errorCheck = details.find_all('dt')[0].text
        print('Site type 2 found.')
    except:
        print('Incompatible Site Tag.')
        return
    
    print('Entering daysOnZillow()')
    days = daysOnZillow(details)
    print('Exiting daysOnZillow()')
    print('Entering zillowExtraData()')
    extra = zillowExtraData(details)
    print('Exiting zillowExtraData()')

    output = {
        'time on zillow':days,
        'house built':extra[0],
        'land':extra[1]
        }
    return output

def daysOnZillow(tag):
    '''
    Need to be careful here as well. Some homes haven't been built yet and so there are no "days" on zillow yet. Instead, this
    scraper will pull the number of views and will not add a unit to the end. Ie. "442" instead of "442" views or "442 days."
    '''
    try:
        days = tag.find('dl',{'class':'hdp__sc-ky7q3p-0 dfePgr'})
        days = days.find_all('dt')[0].text
        print('Tag type 1 found.')
        print(days)
        return days
    except AttributeError:
        print('error 1')
        days = 'ERROR'
    try:
        days = tag.find('dl',{'class':'OverviewStatsComponentsstyles__StyledOverviewStats-sc-1bg6b6d-0 nnBbc'})
        days = days.find_all('dt')[0].text
        print('Tag type 2 found.')
        print(days)
        return days
    except AttributeError:
        print('error 2')
        days = 'ERROR'
    try:
        days = tag.find('dl',{'class':'OverviewStatsComponents__StyledOverviewStats-sc-7d6bsa-0 bOcuTH'})
        days = days.find_all('dt')[0].text
        print('Tag type 3 found.')
        print(days)
        return days
    except AttributeError:
        print('error 3')
        days = 'ERROR'
    
    return days

def zillowExtraData(tag):
    try:
        age = tag.find_all('span',{'class':'Text-c11n-8-84-3__sc-aiai24-0 hdp__sc-6k0go5-3 hrfydd llcOCk'})
        if age == []:
            raise AttributeError
        age = age[1].text
        print('Age: %s' %age)
        land = tag.find_all('span',{'class':'Text-c11n-8-84-3__sc-aiai24-0 hdp__sc-6k0go5-3 hrfydd llcOCk'})
        land = land[2].text
        print(land)
        data = [age, land]
        return data
    except AttributeError:
        print('Error 1')
        data = ['ERROR', 'ERROR']

    try:
        age = tag.find_all('span',{'class':'Text-c11n-8-84-3__sc-aiai24-0 styles__StyledFactValue-sc-6k0go5-3 hrfydd PeHnt'})
        if age == []:
            raise AttributeError
        age = age[1].text
        print(age)
        land = tag.find_all('span',{'class':'Text-c11n-8-84-3__sc-aiai24-0 styles__StyledFactValue-sc-6k0go5-3 hrfydd PeHnt'})
        land = land[2].text
        print(land)
        data = [age, land]
        return data
    except AttributeError:
        print('Error 2')
        data = ['ERROR', 'ERROR']
    
    return data

In [9]:
data = []
for i, url in enumerate(df_housingInfo.url):
    response = requests.get(url=url, cookies=cookies, headers=headers)
    time.sleep(1)
    print('URL #%i. URL: %s' %(i, url))

    if response.status_code != 200:
        print('Zillow has temporarily banned us. Try again soon.')
        break

    content = BeautifulSoup(response.text, 'lxml')
    
    # This needs to correctly deal with the additional data (days, age, land)
    data.append(siteType(content))
    # results.append(data)
    # print(data)
    print('')

df_additional = pd.DataFrame.from_dict(data)
df_housingInfo=pd.concat([df_housingInfo, df_additional], ignore_index=True)

URL #0. URL: https://www.zillow.com/homedetails/4945-Laguna-Vista-Dr-Melbourne-FL-32934/43454629_zpid/
Site type 1 found.
Entering daysOnZillow()
Tag type 1 found.
116 days
Exiting daysOnZillow()
Entering zillowExtraData()
Age: Built in 1974
1.48 Acres
Exiting zillowExtraData()

URL #1. URL: https://www.zillow.com/homedetails/3430-Sandpiper-Ct-Melbourne-FL-32935/66681273_zpid/
Site type 1 found.
Entering daysOnZillow()
error 1
Tag type 2 found.
23 days
Exiting daysOnZillow()
Entering zillowExtraData()
Error 1
Built in 2004
3,920 sqft lot
Exiting zillowExtraData()

URL #2. URL: https://www.zillow.com/homedetails/749-Fernandina-St-NW-Palm-Bay-FL-32907/66682665_zpid/
Site type 1 found.
Entering daysOnZillow()
error 1
Tag type 2 found.
17 days
Exiting daysOnZillow()
Entering zillowExtraData()
Error 1
Built in 2004
10,018 sqft lot
Exiting zillowExtraData()

URL #3. URL: https://www.zillow.com/homedetails/670-Bluefields-St-SE-Palm-Bay-FL-32909/43511910_zpid/
Site type 1 found.
Entering daysO

In [11]:
df_additional = pd.DataFrame.from_dict(data)
df_housingInfo=pd.concat([df_housingInfo, df_additional], axis=1, join='outer', ignore_index=True)
df_housingInfo.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,"2,234",28.143036,-80.708885,"4945 Laguna Vista Dr, Melbourne, FL 32934",https://www.zillow.com/homedetails/4945-Laguna...,"$565,000",4,3,NaN,NaN,NaN,116 days,Built in 1974,1.48 Acres,116 days,Built in 1974,1.48 Acres
1,"1,915",28.161730,-80.651695,"3430 Sandpiper Ct, Melbourne, FL 32935",https://www.zillow.com/homedetails/3430-Sandpi...,"$289,900",3,3,NaN,NaN,NaN,23 days,Built in 2004,"3,920 sqft lot",23 days,Built in 2004,"3,920 sqft lot"
2,"1,398",28.012882,-80.690780,"749 Fernandina St NW, Palm Bay, FL 32907",https://www.zillow.com/homedetails/749-Fernand...,"$285,000",3,2,NaN,NaN,NaN,17 days,Built in 2004,"10,018 sqft lot",17 days,Built in 2004,"10,018 sqft lot"
3,"1,120",27.984524,-80.655075,"670 Bluefields St SE, Palm Bay, FL 32909",https://www.zillow.com/homedetails/670-Bluefie...,"$189,900",3,2,NaN,NaN,NaN,8 days,Built in 1984,0.25 Acres lot,8 days,Built in 1984,0.25 Acres lot
4,"1,984",28.471148,-80.802010,"6425 Stillwater Ave, Cocoa, FL 32927",https://www.zillow.com/homedetails/6425-Stillw...,$125K,4,2,NaN,NaN,NaN,ERROR,ERROR,ERROR,ERROR,ERROR,ERROR


In [15]:
df_housingInfo.to_csv('housingData.csv', sep=',', index=True, encoding='utf-8')